In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-11-27 20:39:33--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2022-11-27 20:39:33 (207 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Books with more than 100 ratings

dfb = df_ratings.groupby(["isbn"]).count().reset_index()
good_books = dfb.loc[dfb["rating"] >= 100]["isbn"]
good_books = df_books.loc[df_books["isbn"].isin(good_books)]

In [ ]:
# Users who have rated more than 200 books
dfu = df_ratings[["user", "rating"]].groupby(["user"]).count().reset_index()
active_users = dfu.loc[dfu["rating"] >= 200]["user"]
df = df_ratings.loc[df_ratings["user"].isin(active_users)]

df = df_ratings.loc[df_ratings["user"].isin(active_users)]
df = df.loc[df["isbn"].isin(good_books["isbn"])]

In [ ]:
# Create a pivot table of book titles as an index

df_temp_pivot = df.pivot(index='isbn',columns='user',values='rating').fillna(0)

title = pd.DataFrame(df_books[df_books.isbn.isin(df_temp_pivot.index)][['isbn','title']]).set_index('isbn')

df_pivot = pd.concat([df_temp_pivot, title], axis=1).set_index('title')

In [ ]:
# Creating and training a model
model = NearestNeighbors(metric='cosine')

model.fit(df_pivot)

NearestNeighbors(metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distance, indice = model.kneighbors(np.array(df_pivot.loc[book]).reshape(-1,int(df_pivot.shape[1])), 6, return_distance=True)
  indice_list = indice.tolist()[0]
  distance_list=distance.tolist()[0]

  book_list = pd.Series(df_pivot.index)[indice.tolist()[0]].values.tolist()
  recommended_books=[]

  for q in range(1,6):
    recommended_books.append([book_list[q], round(distance_list[q], 2)])

  return [book, recommended_books[::-1]]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8], ['The Weight of Water', 0.77], ['The Surgeon', 0.77], ['I Know This Much Is True', 0.77], ['The Lovely Bones: A Novel', 0.72]]]
You passed the challenge! 🎉🎉🎉🎉🎉
